In [8]:
!pip install -q transformers


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

%cd ./drive/MyDrive/Grounding_lm

Mounted at /content/drive


# Data

In [1]:
import pandas as pd

def read_data(dataname):
    df_train = pd.read_csv(f'data/{dataname}/train.csv')
    df_val = pd.read_csv(f'data/{dataname}/validation.csv')
    df_test = pd.read_csv(f'data/{dataname}/test.csv')
    df_test[['text', 'summary']] = df_test[['text', 'summary']].astype(str)
    return df_train, df_val, df_test

In [2]:
df_train, df_val, df_test = read_data(dataname="xsum")
document_text = df_train['text'][0]
document_text

'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but

In [3]:
wikihop_corpus = pd.read_csv('data/wikihop_corpus.csv', dtype=str)
musique_corpus = pd.read_csv('data/musique_corpus.csv', dtype=str)

In [4]:
wikihop_qa = wikihop_corpus.dropna()
musique_qa = musique_corpus.dropna()

# Pre-Hoc

## REALM

In [8]:
import torch
from transformers import AutoTokenizer, RealmForOpenQA, RealmRetriever

retriever = RealmRetriever.from_pretrained("google/realm-orqa-nq-openqa")
tokenizer = AutoTokenizer.from_pretrained("google/realm-orqa-wq-openqa")
model = RealmForOpenQA.from_pretrained("google/realm-orqa-wq-openqa", retriever=retriever)

In [ ]:
question = wikihop_qa['question'][0]
question_ids = tokenizer([question], return_tensors="pt")
answer_ids = tokenizer(
    [wikihop_qa['answer'][0]],
    add_special_tokens=False,
    return_token_type_ids=False,
    return_attention_mask=False,
).input_ids

reader_output, predicted_answer_ids = model(**question_ids, answer_ids=answer_ids, return_dict=False)
predicted_answer = tokenizer.decode(predicted_answer_ids)
loss = reader_output.loss